---
title: Data Import from WorldPop Population Data
short_title: Population Data Import
---

In this notebook...

----------------------------------------
## Requirements

### 1. Connect to DHIS2

In order to run this notebook, you first need to connect to an instance of DHIS2. For our example, we will connect to a local instance of DHIS2 containing the standard Sierra Leone demo database, but you should be able to switch out the instance url and credentials to work directly with your own database. 

In [2]:
from dhis2_client import DHIS2Client
from dhis2_client.settings import ClientSettings

# Create DHIS2 client connection
cfg = ClientSettings(
  base_url="http://localhost:8080",
  username="admin",
  password="district"
)
client = DHIS2Client(settings=cfg)

# Verify connection
info = client.get_system_info()
print("Current DHIS2 version:", info["version"])

Current DHIS2 version: 2.42.2


### 2. DHIS2 data element

We also need the data element for importing population data into. For now, we use the existing population data element of the Sierra Leone dummy database. To make this work with your local instance, replace the `data_element_id` below: 

In [2]:
data_element_id = 'WUg3MYWQ7pt'

-------------
## Workflow for importing WorldPop population data

In [4]:
import dhis2eo
import dhis2eo.org_units
import dhis2eo.data.worldpop
import dhis2eo.utils.aggregate

### Step 1: Retrieve organisation units

First we get the organisation units from DHIS2 and load them into a format we can work with: 

In [ ]:
org_units_geojson = client.get_org_units_geojson(level=2)
org_units = dhis2eo.org_units.from_dhis2_geojson(org_units_geojson)
org_units

,org_unit_id,name,level,geometry
0,O6uvpzGd5pu,Bo,2,"POLYGON ((-11.5914 8.4875, -11.5906 8.4769, -1..."
1,fdc6uOvgoji,Bombali,2,"POLYGON ((-11.8091 9.2032, -11.8102 9.1944, -1..."
2,lc3eMKXaEfw,Bonthe,2,"MULTIPOLYGON (((-12.5568 7.3832, -12.5574 7.38..."
3,jUb8gELQApl,Kailahun,2,"POLYGON ((-10.7972 7.5866, -10.8002 7.5878, -1..."
4,PMa2VCrupOd,Kambia,2,"MULTIPOLYGON (((-13.1349 8.8471, -13.1343 8.84..."
5,kJq2mPyFEHo,Kenema,2,"POLYGON ((-11.3596 8.5317, -11.3513 8.5234, -1..."
6,qhqAxPSTUXp,Koinadugu,2,"POLYGON ((-10.585 9.0434, -10.5877 9.0432, -10..."
7,Vth0fbpFcsO,Kono,2,"POLYGON ((-10.585 9.0434, -10.5848 9.0432, -10..."
8,jmIPBj66vD6,Moyamba,2,"MULTIPOLYGON (((-12.6351 7.6613, -12.6346 7.66..."
9,TEQlaapDQoK,Port Loko,2,"MULTIPOLYGON (((-13.119 8.4718, -13.1174 8.470..."


### Step 2: Download WorldPop data

We provide a convenience function for downloading WorldPop historical and projected population data from the [WorldPop v2 dataset between 2015 and 2030](https://hub.worldpop.org/project/categories?id=3). Let's get the projected data for 2030:

In [5]:
year = 2030
country_code = 'SLE'
data = dhis2eo.data.worldpop.get_population_data(year, country_code)
print(f'Population data {data}')

dhis2eo.data.worldpop - INFO - Loading from cache: C:\Users\karimba\AppData\Local\Temp\sle_pop_2030_CN_100m_R2025A_v1.tif
Population data <xarray.Dataset> Size: 54MB
Dimensions:    (x: 3635, y: 3695)
Coordinates:
  * x          (x) float64 29kB -13.3 -13.3 -13.3 -13.3 ... -10.27 -10.27 -10.27
  * y          (y) float64 30kB 10.0 9.999 9.998 9.997 ... 6.923 6.922 6.921
    year       int64 8B 2030
Data variables:
    total_pop  (y, x) float32 54MB ...
Attributes:
    TIFFTAG_DOCUMENTNAME:      sle_pop_2030_CN_100m_R2025A_v1
    TIFFTAG_IMAGEDESCRIPTION:  SLE population 2030 [WorldPop R2025A v1]
    TIFFTAG_DATETIME:          2025-07-29 04:37:30
    TIFFTAG_COPYRIGHT:         CC-BY-4.0
    Description:               SLE population 2030 [WorldPop R2025A v1]
    AREA_OR_POINT:             Area


### Step 3: Aggregate the data to organisation units

The next step is using the `aggregate.to_org_units` convenience function to aggregate the climate data to a set of input organisation units. Let's try it for our previously downloaded test data:

In [ ]:
variables = {'total_pop': 'sum'}
agg = dhis2eo.utils.aggregate.to_org_units(data, org_units, variables=variables)
print(agg)

    org_unit_id  year     total_pop
0   O6uvpzGd5pu  2030  1.050703e+06
1   fdc6uOvgoji  2030  7.419016e+05
2   lc3eMKXaEfw  2030  4.229886e+05
3   jUb8gELQApl  2030  7.207222e+05
4   PMa2VCrupOd  2030  4.757711e+05
5   kJq2mPyFEHo  2030  1.011293e+06
6   qhqAxPSTUXp  2030  4.814348e+05
7   Vth0fbpFcsO  2030  8.176847e+05
8   jmIPBj66vD6  2030  4.530076e+05
9   TEQlaapDQoK  2030  8.062628e+05
10  bL4ooGhyHRQ  2030  5.417854e+05
11  eIQbndfxQMb  2030  6.943592e+05
12  at6UHUQatSo  2030  1.387464e+06


### Step 4: Import yearly population data into DHIS2

Finally, let's combine these steps to download, aggregate, and import data to DHIS2 for a range of recent years available in the WorldPop dataset: 

In [ ]:
from dhis2eo.integrations.pandas import dataframe_to_dhis2_json
for year in range(2015, 2025+1):
    print(f'Year: {year}')
    # Download data...
    data = dhis2eo.data.worldpop.get_population_data(year, country_code)
    # Aggregate data...
    variables = {'total_pop': 'sum'}
    agg = dhis2eo.utils.aggregate.to_org_units(data, org_units, variables=variables)
    # Convert and import to DHIS2...
    payload = dataframe_to_dhis2_json(
        df=agg,
        org_unit_col='org_unit_id',
        period_col='year',
        value_col='total_pop',
        data_element_id=data_element_id,
    )
    print('Importing to DHIS2...')
    res = client.post("/api/dataValueSets", json=payload)
    print("Results:", res['response']['importCount'])

Year: 2015
dhis2eo.data.worldpop - INFO - Loading from cache: C:\Users\karimba\AppData\Local\Temp\sle_pop_2015_CN_100m_R2025A_v1.tif
Importing to DHIS2...
Results: {'imported': 0, 'updated': 0, 'ignored': 13, 'deleted': 0}
Year: 2016
dhis2eo.data.worldpop - INFO - Loading from cache: C:\Users\karimba\AppData\Local\Temp\sle_pop_2016_CN_100m_R2025A_v1.tif
Importing to DHIS2...
Results: {'imported': 0, 'updated': 0, 'ignored': 13, 'deleted': 0}
Year: 2017
dhis2eo.data.worldpop - INFO - Loading from cache: C:\Users\karimba\AppData\Local\Temp\sle_pop_2017_CN_100m_R2025A_v1.tif
Importing to DHIS2...
Results: {'imported': 0, 'updated': 0, 'ignored': 13, 'deleted': 0}
Year: 2018
dhis2eo.data.worldpop - INFO - Loading from cache: C:\Users\karimba\AppData\Local\Temp\sle_pop_2018_CN_100m_R2025A_v1.tif
Importing to DHIS2...
Results: {'imported': 0, 'updated': 0, 'ignored': 13, 'deleted': 0}
Year: 2019
dhis2eo.data.worldpop - INFO - Loading from cache: C:\Users\karimba\AppData\Local\Temp\sle_pop_20